In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [ ]:
train_NaTransport = train.drop("Transported", axis=1)
features = pd.concat([train_NaTransport, test])

In [ ]:
features = features.sort_values('Age')
msno.matrix(features)

In [ ]:


sns.pairplot(data=train, kind="scatter", hue="Transported", diag_kind="auto", corner=True, 
            x_vars=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age'],
            y_vars=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age']);



## The UMAP below describes absolutely nothing, it has every meaning to nothing.

In [ ]:
import umap

train_dropna = train.dropna(axis=0)
features_dropna = features.dropna(axis=0)
# Extract the features (numerical columns) from the DataFrame
# Replace 'Category' with the actual column name you want to use for coloring points
train_filt = train_dropna[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Age', 'Transported']]

# Initialize UMAP reducer
reducer = umap.UMAP()

# Apply UMAP dimensionality reduction to the features
embedding = reducer.fit_transform(train_filt)

# Create a scatterplot of the reduced-dimensional data
plt.figure(figsize=(8, 6))
plt.scatter(embedding[:, 0], embedding[:, 1], c=pd.factorize(train_dropna['Transported'])[0], cmap='viridis', s=50)
plt.title('UMAP Visualization')
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.colorbar()
plt.show()